## Historical people: Quick and dirty

This example shows how to get some initial record linkage results as quickly as possible.

There are many ways to improve the accuracy of this model. But this may be a good place to start if you just want to give Splink a try and see what it's capable of.


<a target="_blank" href="https://colab.research.google.com/github/moj-analytical-services/splink/blob/splink4_examples_notebooks/docs/demos/examples/duckdb/quick_and_dirty_persons.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


In [1]:
# Uncomment and run this cell if you're running in Google Colab.
# !pip install git+https://github.com/moj-analytical-services/splink.git@splink4_examples_notebooks

In [2]:
from splink.datasets import splink_datasets

df = splink_datasets.historical_50k
df.head(5)

,unique_id,cluster,full_name,first_and_surname,first_name,surname,dob,birth_place,postcode_fake,gender,occupation
0,Q2296770-1,Q2296770,"thomas clifford, 1st baron clifford of chudleigh",thomas chudleigh,thomas,chudleigh,1630-08-01,devon,tq13 8df,male,politician
1,Q2296770-2,Q2296770,thomas of chudleigh,thomas chudleigh,thomas,chudleigh,1630-08-01,devon,tq13 8df,male,politician
2,Q2296770-3,Q2296770,tom 1st baron clifford of chudleigh,tom chudleigh,tom,chudleigh,1630-08-01,devon,tq13 8df,male,politician
3,Q2296770-4,Q2296770,thomas 1st chudleigh,thomas chudleigh,thomas,chudleigh,1630-08-01,devon,tq13 8hu,None,politician
4,Q2296770-5,Q2296770,"thomas clifford, 1st baron chudleigh",thomas chudleigh,thomas,chudleigh,1630-08-01,devon,tq13 8df,None,politician


In [3]:
from splink.blocking_rule_library import block_on
import splink.comparison_library as cl
import splink.comparison_template_library as ctl

settings = {
    "link_type": "dedupe_only",
    "blocking_rules_to_generate_predictions": [
        block_on("full_name"),
        block_on("substr(full_name,1,6)", "dob", "birth_place"),
        block_on("dob", "birth_place"),
        block_on("postcode_fake"),
    ],
    "comparisons": [
        cl.JaroWinklerAtThresholds("full_name", [0.9, 0.7]).configure(
            term_frequency_adjustments=True
        ),
        ctl.DateComparison(
            "dob",
            input_is_string=True,
            datetime_metrics=["day", "month", "year"],
            datetime_thresholds=[5, 1, 5],

        ),
        cl.LevenshteinAtThresholds("postcode_fake", 2),
        cl.JaroWinklerAtThresholds("birth_place", 0.9).configure(
            term_frequency_adjustments=True
        ),
        cl.ExactMatch("occupation").configure(term_frequency_adjustments=True),
    ],
}

In [4]:
from splink.linker import Linker
from splink.database_api import DuckDBAPI

linker = Linker(df, settings, database_api=DuckDBAPI(), set_up_basic_logging=False)
deterministic_rules = [
    "l.full_name = r.full_name",
    "l.postcode_fake = r.postcode_fake and l.dob = r.dob",
]

linker.estimate_probability_two_random_records_match(deterministic_rules, recall=0.6)

In [5]:
linker.estimate_u_using_random_sampling(max_pairs=2e6)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [6]:
results = linker.predict(threshold_match_probability=0.9)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


 -- WARNING --
You have called predict(), but there are some parameter estimates which have neither been estimated or specified in your settings dictionary.  To produce predictions the following untrained trained parameters will use default values.
Comparison: 'full_name':
    m values not fully trained
Comparison: 'dob':
    m values not fully trained
Comparison: 'postcode_fake':
    m values not fully trained
Comparison: 'birth_place':
    m values not fully trained
Comparison: 'occupation':
    m values not fully trained


In [7]:
results.as_pandas_dataframe(limit=5)

,match_weight,match_probability,unique_id_l,unique_id_r,full_name_l,full_name_r,gamma_full_name,dob_l,dob_r,gamma_dob,postcode_fake_l,postcode_fake_r,gamma_postcode_fake,birth_place_l,birth_place_r,gamma_birth_place,occupation_l,occupation_r,gamma_occupation,match_key
0,3.171715,0.900112,Q3285335-16,Q3285335-8,mancherji bhownagri,sir mancherji bhownagri,1,1857-08-15,1851-08-12,0,dn37 0fd,dn37 0fd,2,mumbai,mumbai,2,None,None,-1,3
1,3.173155,0.900201,Q105737087-17,Q105737087-5,lady of little breakfast),emily trevenen,0,1786-85-11,None,-1,co12 3rs,co12 3rs,2,tendring,tendring,2,None,None,-1,3
2,3.173155,0.900201,Q105737087-17,Q105737087-6,lady of little breakfast),emily trevenen,0,1786-85-11,None,-1,co12 3rs,co12 3rs,2,tendring,tendring,2,None,None,-1,3
3,3.173155,0.900201,Q105737087-2,Q105737087-5,lady (author of derwent's emily breakfast),emily trevenen,0,1786-05-11,None,-1,co12 3rs,co12 3rs,2,tendring,tendring,2,None,None,-1,3
4,3.173155,0.900201,Q105737087-2,Q105737087-6,lady (author of derwent's emily breakfast),emily trevenen,0,1786-05-11,None,-1,co12 3rs,co12 3rs,2,tendring,tendring,2,None,None,-1,3
